# Notebook Land --> Bronze

## Configuração dos schemas e volume na landing:

In [0]:
# Primeiro a criação do catálogo medalhao_case
spark.sql(f"CREATE CATALOG IF NOT EXISTS medalhao_case")
spark.sql(f"USE CATALOG medalhao_case")

# Depois criei os schemas
spark.sql(f"CREATE SCHEMA IF NOT EXISTS bronze")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS silver")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS gold")

# E por último o volume na landing
spark.sql(f"CREATE VOLUME IF NOT EXISTS default.landing")

## Função para a ingestão dos dados do case:

In [0]:
from pyspark.sql import functions as F
from pyspark.sql import SparkSession

def ingestao_dados(nome_arquivo, nome_tabela_bronze):
    try:
        caminho_landing = f"/Volumes/medalhao_case/default/landing/{nome_arquivo}"
        
        # Leitura do arquivo passado como parâmetro de entrada.
        df = spark.read.csv(caminho_landing, header=True, inferSchema=True)
        
        # Aqui verifico de o arquivo não está vazio.
        if df.count() == 0:
            raise ValueError(f"O arquivo {nome_arquivo} está vazio.")

        # Adicionei a coluna 'hora_da_ingestao' mostrando hora/data de ingestão dos dados.
        df_com_metadata = df.withColumn("hora_da_ingestao", F.current_timestamp())

        # Escrita no formato Delta e sobrescre se já existir.
        spark.sql('drop table if exists bronze.{nome_tabela_bronze}')
        df_com_metadata.write.format("delta").mode("overwrite").saveAsTable(f"bronze.{nome_tabela_bronze}")
        
    #Aqui é apenas para filtrar possíveis erros
    except Exception as e:
        print(f"Erro ao processar {nome_arquivo}: {str(e)}")

In [0]:
# Organizei em um dicionário os arquivos e tabelas pra ficar mais fácil de fazer a ingestão.
arquivos_e_tabelas = {
    "Chamados_Hora.CSV": "bronze_chamados_hora",
    "base_atendentes.csv": "bronze_base_atendentes",
    "base_canais.csv": "bronze_base_canais",
    "base_motivos.csv": "bronze_base_motivos",
    "chamados.csv": "bronze_chamados",
    "clientes.csv": "bronze_clientes",
    "custos.csv": "bronze_custos",
    "pesquisa_satisfacao.csv": "bronze_pesquisa_satisfacao"
}

# Aqui eu chamo a função ingestão de dados para os arquivos e tabelas do dicionário.
for arquivo, tabela in arquivos_e_tabelas.items():
    ingestao_dados(arquivo, tabela)